# Custom Display Logic

## Overview

As described in the [Rich Output](Rich Output.ipynb) tutorial, the IPython display system can display rich representations of objects in the following formats:

* JavaScript
* HTML
* PNG
* JPEG
* SVG
* LaTeX
* PDF

This Notebook shows how you can add custom display logic to your own classes, so that they can be displayed using these rich representations. There are two ways of accomplishing this:

1. Implementing special display methods such as `_repr_html_` when you define your class.
2. Registering a display function for a particular existing class.

This Notebook describes and illustrates both approaches.

Import the IPython display functions.

In [ ]:
from IPython.display import (
    display, display_html, display_png, display_svg
)

Parts of this notebook need the matplotlib inline backend:

The formatter object has a couple of methods for registering custom display functions for existing types.

In [ ]:
help(latex_f.for_type)

In [ ]:
help(latex_f.for_type_by_name)

In this case, we will use `for_type_by_name` to register `poly_to_latex` as the display function for the `Polynomial` type:

In [ ]:
latex_f.for_type_by_name('numpy.polynomial.polynomial',
                                 'Polynomial', poly_to_latex)

Once the custom display function has been registered, all NumPy `Polynomial` instances will be represented by their LaTeX form instead:

In [ ]:
p

In [ ]:
p2 = np.polynomial.Polynomial([-20, 71, -15, 1])
p2

## Custom Mimetypes with `_repr_mimebundle_`

Available on IPython 5.4+ and 6.1+.

For objects needing full control over the `repr` protocol may decide to implement the `_repr_mimebundle_(include, exclude)` method.
Unlike the other `_repr_*_` methods must return many representation of the object in a mapping object which keys are _mimetypes_ and value are associated data. The `_repr_mimebundle_()` method, may also return a second mapping from _mimetypes_ to metadata. 

Example:

In [ ]:
class Gaussian(object):
    """A simple object holding data sampled from a Gaussian distribution.
    """
    def __init__(self, mean=0.0, std=1, size=1000):
        self.data = np.random.normal(mean, std, size)
        self.mean = mean
        self.std = std
        self.size = size
        # For caching plots that may be expensive to compute
        self._png_data = None
        
    def _figure_data(self, format):
        fig, ax = plt.subplots()
        ax.hist(self.data, bins=50)
        ax.set_xlim(-10.0,10.0)
        data = print_figure(fig, format)
        # We MUST close the figure, otherwise IPython's display machinery
        # will pick it up and send it as output, resulting in a double display
        plt.close(fig)
        return data
    
    def _compute_mathml(self):
        return """
        <math xmlns="http://www.w3.org/1998/Math/MathML">
          <mrow class="MJX-TeXAtom-ORD">
            <mi class="MJX-tex-caligraphic" mathvariant="script">N</mi>
          </mrow>
          <mo stretchy="false">(</mo>
          <mi>&#x03BC;<!-- μ --></mi>
          <mo>=</mo>
          <mn>{mu}</mn>
          <mo>,</mo>
          <mi>&#x03C3;<!-- σ --></mi>
          <mo>=</mo>
          <mn>{sigma}</mn>
          <mo stretchy="false">)</mo>
          <mo>,</mo>
          <mtext>&#xA0;</mtext>
          <mi>N</mi>
          <mo>=</mo>
          <mn>{N}</mn>
        </math>
        """.format(N=self.size, mu=self.mean, sigma=self.std)
        
    def _repr_mimebundle_(self, include, exclude, **kwargs):
        """
        repr_mimebundle shoudl accept include, exclude and **kwargs
        """
        if self._png_data is None:
            self._png_data = self._figure_data('png')
        math = r'$\mathcal{N}(\mu=%.2g, \sigma=%.2g),\ N=%d$' % (self.mean,
                                                                 self.std, self.size)
        data = {'image/png':self._png_data,
                'text/latex':math,
                'application/mathml+xml': self._compute_mathml()
                }
        if include:
            data = {k:v for (k,v) in data.items() if k in include}
        if exclude:
            data = {k:v for (k,v) in data.items() if k not in exclude}
        return data

In [ ]:
# that is deffinitively wrong as it shoudl show the PNG. 
display(Gaussian())

In the above example, the 3 mimetypes are embeded in the notebook document this allowing custom extensions and converters to display the representation(s) of their choice.

For example, converting this noetebook to _epub_ may decide to use the MathML representation as most ebook reader cannot run mathjax (unlike browsers). 


### Implementation guidelines

The `_repr_mimebundle_` methods is also given two keywords parameters :  `include` and `exclude`. Each can be a  containers (e.g.:`list`, `set` ...) of mimetypes to return or `None`, This allows implementation to avoid computing potentially unnecessary and expensive mimetypes representations. 

When `include` is non-empty (empty `list` or None), `_repr_mimebundle_` may decide to returns only the mimetypes in include.
When `exclude` is non-empty, `_repr_mimebundle_` may decide to not return any mimetype in exclude. 
If both `include` and `exclude` and overlap, mimetypes present in exclude may not be returned. 

If implementations decide to ignore the `include` and `exclude` logic and always returns a full mimebundles, the IPython kernel will take care of removing non-desired representations.

The `_repr_mimebundle_` method should accept arbitrary keyword arguments for future compatiility.
